In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## चरण १: संरचित आउटपुटका लागि Pydantic मोडेलहरू परिभाषित गर्नुहोस्

यी मोडेलहरूले एजेन्टहरूले फर्काउने **schema** परिभाषित गर्छन्। `response_format` लाई Pydantic सँग प्रयोग गर्दा सुनिश्चित हुन्छ:
- ✅ प्रकार-सुरक्षित डेटा निकाल्ने प्रक्रिया
- ✅ स्वचालित मान्यता
- ✅ स्वतन्त्र-पाठ प्रतिक्रियाबाट कुनै पार्सिङ त्रुटि हुँदैन
- ✅ फिल्डहरूमा आधारित सजिलो सर्तात्मक रुटिङ


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## चरण २: होटल बुकिङ उपकरण बनाउनुहोस्

यो उपकरण **availability_agent** ले कोठा उपलब्ध छन् कि छैनन् भनेर जाँच गर्न कल गर्नेछ। हामी `@ai_function` डेकोरेटर प्रयोग गर्छौं ताकि:
- पायथन फङ्सनलाई AI-कल गर्न मिल्ने उपकरणमा परिवर्तन गर्न सकियोस्
- LLM का लागि स्वत: JSON स्किमा निर्माण गर्न सकियोस्
- प्यारामिटरको मान्यता जाँच गर्न सकियोस्
- एजेन्टहरूले स्वत: कल गर्न सक्ने बनाइयोस्

यस डेमोका लागि:
- **स्टकहोम, सिएटल, टोकियो, लन्डन, एम्स्टर्डम** → कोठा उपलब्ध छन् ✅
- **अन्य सबै शहरहरू** → कोठा उपलब्ध छैन ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## चरण ३: रुटिङका लागि सर्त कार्यहरू परिभाषित गर्नुहोस्

यी कार्यहरूले एजेन्टको प्रतिक्रिया जाँच गर्छन् र कार्यप्रवाहमा कुन बाटो लिनु पर्ने निर्धारण गर्छन्।

**मुख्य ढाँचा:**
1. सन्देश `AgentExecutorResponse` हो कि होइन जाँच गर्नुहोस्
2. संरचित आउटपुट (Pydantic मोडेल) पार्स गर्नुहोस्
3. रुटिङ नियन्त्रण गर्न `True` वा `False` फर्काउनुहोस्

कार्यप्रवाहले **किनारहरू** मा यी सर्तहरू मूल्याङ्कन गरेर कुन कार्यकारीलाई अर्कोमा बोलाउनुपर्ने निर्णय गर्नेछ।


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## चरण ४: कस्टम डिस्प्ले एक्सिक्युटर बनाउनुहोस्

एक्सिक्युटरहरू वर्कफ्लोका घटकहरू हुन् जसले रूपान्तरण वा साइड इफेक्टहरू प्रदर्शन गर्छन्। हामी `@executor` डेकोरेटर प्रयोग गरेर अन्तिम नतिजा देखाउने कस्टम एक्सिक्युटर बनाउँछौं।

**मुख्य अवधारणाहरू:**
- `@executor(id="...")` - कार्यलाई वर्कफ्लो एक्सिक्युटरको रूपमा दर्ता गर्छ
- `WorkflowContext[Never, str]` - इनपुट/आउटपुटका लागि टाइप संकेतहरू
- `ctx.yield_output(...)` - अन्तिम वर्कफ्लो नतिजा प्रदान गर्छ


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## चरण ५: वातावरणीय भेरिएबलहरू लोड गर्नुहोस्

LLM क्लाइन्ट कन्फिगर गर्नुहोस्। यो उदाहरण निम्नसँग काम गर्दछ:
- **GitHub Models** (GitHub टोकनको साथ निःशुल्क स्तर)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## चरण ६: संरचित नतिजासहित AI एजेन्टहरू बनाउनुहोस्

हामी **तीन विशेष एजेन्टहरू** बनाउँछौं, प्रत्येकलाई `AgentExecutor` मा समेटेर:

1. **availability_agent** - उपकरण प्रयोग गरेर होटलको उपलब्धता जाँच गर्दछ
2. **alternative_agent** - वैकल्पिक शहरहरूको सुझाव दिन्छ (जब कोठा उपलब्ध छैन)
3. **booking_agent** - बुकिङको प्रोत्साहन दिन्छ (जब कोठा उपलब्ध छ)

**मुख्य विशेषताहरू:**
- `tools=[hotel_booking]` - एजेन्टलाई उपकरण प्रदान गर्दछ
- `response_format=PydanticModel` - संरचित JSON नतिजा सुनिश्चित गर्दछ
- `AgentExecutor(..., id="...")` - कार्यप्रवाहमा प्रयोगको लागि एजेन्टलाई समेट्छ


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## चरण ७: सशर्त किनाराहरू सहित कार्यप्रवाह निर्माण गर्नुहोस्

अब हामी `WorkflowBuilder` प्रयोग गरेर सशर्त मार्गनिर्देशनसहितको ग्राफ निर्माण गर्छौं:

**कार्यप्रवाह संरचना:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**मुख्य विधिहरू:**
- `.set_start_executor(...)` - प्रवेश बिन्दु सेट गर्दछ
- `.add_edge(from, to, condition=...)` - सशर्त किनारा थप्छ
- `.build()` - कार्यप्रवाहलाई अन्तिम रूप दिन्छ


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## चरण ८: परीक्षण केस १ चलाउनुहोस् - उपलब्धता बिना सहर (पेरिस)

आउनुहोस्, **उपलब्धता नभएको** मार्ग परीक्षण गरौं, पेरिसमा होटलहरू अनुरोध गरेर (जसमा हाम्रो सिमुलेशनमा कोठाहरू छैनन्)।


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## चरण ९: परीक्षण केस २ चलाउनुहोस् - उपलब्धता भएको सहर (स्टकहोम)

अब हामी **उपलब्धता** मार्ग परीक्षण गर्नेछौं, स्टकहोममा होटलहरू अनुरोध गरेर (जसमा हाम्रो सिमुलेशनमा कोठाहरू उपलब्ध छन्)।


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## मुख्य बुँदाहरू र आगामी कदमहरू

### ✅ तपाईंले सिक्नुभएको:

1. **WorkflowBuilder ढाँचा**
   - `.set_start_executor()` प्रयोग गरेर प्रवेश बिन्दु परिभाषित गर्नुहोस्
   - `.add_edge(from, to, condition=...)` प्रयोग गरेर सर्त अनुसार रुटिङ गर्नुहोस्
   - `.build()` कल गरेर कार्यप्रवाहलाई अन्तिम रूप दिनुहोस्

2. **सर्त अनुसार रुटिङ**
   - सर्त कार्यहरू `AgentExecutorResponse` निरीक्षण गर्छन्
   - संरचित आउटपुटलाई पार्स गरेर रुटिङ निर्णय लिन्छन्
   - एज सक्रिय गर्न `True` फिर्ता गर्नुहोस्, स्किप गर्न `False` फिर्ता गर्नुहोस्

3. **टूल एकीकरण**
   - Python कार्यहरूलाई AI टूलमा बदल्न `@ai_function` प्रयोग गर्नुहोस्
   - एजेन्टहरूले आवश्यक पर्दा टूलहरू स्वचालित रूपमा कल गर्छन्
   - टूलहरूले JSON फिर्ता गर्छन् जसलाई एजेन्टहरूले पार्स गर्न सक्छन्

4. **संरचित आउटपुटहरू**
   - प्रकार-सुरक्षित डाटा निकाल्न Pydantic मोडेलहरू प्रयोग गर्नुहोस्
   - एजेन्टहरू सिर्जना गर्दा `response_format=MyModel` सेट गर्नुहोस्
   - `Model.model_validate_json()` प्रयोग गरेर प्रतिक्रियाहरू पार्स गर्नुहोस्

5. **कस्टम Executors**
   - कार्यप्रवाह घटकहरू सिर्जना गर्न `@executor(id="...")` प्रयोग गर्नुहोस्
   - Executors ले डाटा परिवर्तन गर्न वा साइड इफेक्टहरू प्रदर्शन गर्न सक्छन्
   - कार्यप्रवाह परिणाम उत्पादन गर्न `ctx.yield_output()` प्रयोग गर्नुहोस्

### 🚀 वास्तविक जीवनका प्रयोगहरू:

- **यात्रा बुकिङ**: उपलब्धता जाँच गर्नुहोस्, विकल्पहरू सुझाव दिनुहोस्, तुलना गर्नुहोस्
- **ग्राहक सेवा**: समस्या प्रकार, भावना, प्राथमिकताका आधारमा रुट गर्नुहोस्
- **ई-कमर्स**: स्टक जाँच गर्नुहोस्, विकल्पहरू सुझाव दिनुहोस्, अर्डर प्रक्रिया गर्नुहोस्
- **सामग्री नियन्त्रण**: विषाक्तता स्कोर, प्रयोगकर्ता झण्डाका आधारमा रुट गर्नुहोस्
- **स्वीकृति कार्यप्रवाहहरू**: रकम, प्रयोगकर्ता भूमिका, जोखिम स्तरका आधारमा रुट गर्नुहोस्
- **बहु-चरण प्रशोधन**: डाटा गुणस्तर, पूर्णताका आधारमा रुट गर्नुहोस्

### 📚 आगामी कदमहरू:

- थप जटिल सर्तहरू थप्नुहोस् (एकाधिक मापदण्ड)
- कार्यप्रवाह अवस्था व्यवस्थापनको साथ लूपहरू कार्यान्वयन गर्नुहोस्
- पुन: प्रयोग गर्न मिल्ने घटकहरूको लागि उप-कार्यप्रवाहहरू थप्नुहोस्
- वास्तविक API हरूसँग एकीकृत गर्नुहोस् (होटल बुकिङ, स्टक प्रणालीहरू)
- त्रुटि व्यवस्थापन र फिर्ता मार्गहरू थप्नुहोस्
- निर्मित दृश्यात्मक उपकरणहरूको साथ कार्यप्रवाहहरू दृश्यात्मक बनाउनुहोस्



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव शुद्धताको लागि प्रयास गर्छौं, तर कृपया जानकार रहनुहोस् कि स्वचालित अनुवादहरूमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
